<a href="https://colab.research.google.com/github/searchsolved/search-solved-public-seo/blob/main/best_selling_products_to_xml_sitemap/best_selling_products_to_xml_sitemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert Best Selling Products (Transactions or Revenue) to An XML Sitemap
Why? Uploading an XML sitemap allows you to retrieve detailed Search Coverage data for your best performing URLs).

This means you can spot and resolve any indexing issues and make more money. (e.g. it could be that your products dominate adwords, but other issues are holding it back from organic). 

## How to use:

1.   Download a Landing Page Report (In Excel Format) from Google Analytics (Behaviour > Site Content > Landing Pages).
2.   Specify the domain name in the cell below
3.   Run all cells from the Runtime menu above and upload the Analytics export when prompted.

## Options:

*   Specify the domain of your Website. (Required)
*   Specify Top X Percent of Transactions / Revenue
*   Choice of Transactions of Revenue

## Output
*   XML Sitemap - ready to upload to Search Console.

# Set All Options Below - Make Sure to Correctly Set Your Domain!

In [ ]:
domain = "https://www.example.com"
drop_words = "checkout|basket|paypal|search|account|(not set)"
select_on = "Transactions"
#select_on = "Revenue"
top_percent = 5  # set the top X percent to keep

In [ ]:
!pip install pandas

In [ ]:
import urllib.parse as urlparse
from glob import glob

import pandas as pd
from jinja2 import Template
from google.colab import files

In [ ]:
# Upload a Landing Page Report from GA - Must be an Excel File Export - csv not accepted!
uploaded = files.upload()

In [ ]:
filename = list(uploaded.keys())[0]  # get the filename from the upload
df = pd.read_excel(filename, 'Dataset1')  # choose the right sheet from Excel

In [ ]:
# drop NaNs, force all values to str, append the domain and parse out the urls
df = df[df["Landing Page"].notna()]
df['Landing Page'] = df['Landing Page'].astype(str)
df['Landing Page'] = domain + (df['Landing Page'])
df["protocol"], df["domain"], df["path"], df["query"], df["fragment"] = zip(*df["Landing Page"].map(urlparse.urlsplit))
df['path'] = df['path'].str.split('&').str[0]  #  Split / remove anything with an & symbol

In [ ]:
# calculate top x percentage of rows and drop anything below threshold
count_rows = df.shape[0]
df[select_on] = df[select_on].astype(int)
df = df.sort_values(by=select_on, ascending=False)
drop_rows = top_percent * count_rows / 100
drop_rows = int(drop_rows)
print("Creating XML Sitemap for top", drop_rows, "Rows by", select_on)
df = df[df.index < drop_rows]

In [ ]:
# drop urls that match the drop_words value. (Useful to block checkout, baskets URLs etc) - then drop duplicates
df = df[~df["path"].isin(["/"])]
df = df[~df["path"].str.contains(drop_words, na=False)]
df.drop_duplicates(subset="path", inplace=True)

In [ ]:
# get today's date and add to new column (used for <lastmod>)
df['Date'] = pd.date_range('today', periods=len(df), freq='D').normalize()
df['path'] = domain + df['path']  # append the domain in before creating the sitemap

In [ ]:
# make a simple df with two column values for itertuples
df_sitemap = df[['path', 'Date']]
df_sitemap = df_sitemap.set_index(['path'])

In [ ]:
# make the sitemap and print a sample of the output
sitemap_template = """<?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns = "http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.sitemaps.org/schemas/sitemap/0.9 http://www.sitemaps.org/schemas/sitemap/0.9/sitemap.xsd">
  {% for page in pages %}
  <url>
    <loc>{{page[0]|safe}}</loc>
    <lastmod>{{page[1]}}</lastmod>
    <priority>0.80</priority>
    
  </url>
  {% endfor %}
</urlset>"""

template = Template(sitemap_template)
xml_sitemap_output = template.render(pages=df_sitemap.itertuples())

print(xml_sitemap_output[:500])


In [ ]:
with open("high_value_landing_pages.xml", "w") as fh:
    fh.write(xml_sitemap_output)
files.download("high_value_landing_pages.xml")